In [5]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [6]:
# requests.get("https://www.ambitionbox.com/list-of-companies?campaign=desktop_nav",headers=headers).text
# when above code gives error due to website not allowed

# When reponse code is [403]:

In [7]:
headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
webpage = requests.get("https://www.ambitionbox.com/list-of-companies?campaign=desktop_nav",headers=headers).text

In [8]:
soup = BeautifulSoup(webpage,'lxml')

# Approach 1: fetch the data individually
### problem in this approach: if there is incosistency in the data like some data is missing then there could be mismatch with other datapoints

In [ ]:
# print(soup.prettify()) # format html code in right way

In [ ]:
# soup.find_all('h1')[0].text.strip()

In [ ]:
# name = []
# for i in soup.find_all('h2', class_="companyCardWrapper__companyName"):
#     name.append(i.text.strip())

In [ ]:
# ratings = []
# for i in soup.find_all('span', class_="companyCardWrapper__companyRatingValue"):
#     ratings.append(i.text.strip())

In [ ]:
# ratings_count = []
# for i in soup.find_all('span', class_="companyCardWrapper__ActionCount"):
#     ratings_count.append(i.text.strip())

In [ ]:
# info = []
# for i in soup.find_all('span', class_="companyCardWrapper__interLinking"):
#     info.append(i.text.strip())

In [ ]:
# type = [info[i].split('|')[0] for i in range(len(info))]
# size = [info[i].split('|')[1] for i in range(len(info))]
# type = [info[i].split('|')[2] for i in range(len(info))]
# oldness = [info[i].split('|')[3] for i in range(len(info))]
# city = [info[i].split('|')[4] for i in range(len(info)-2)]

# Approach 2: Through looping:
#### where the data is missing we will put NaN

In [9]:
company = soup.find_all('div', class_="companyCardWrapper")

In [10]:
name = []
rating = []
reviews = []
type = []
size = []
public_private = []
how_old = []
city = []
positive_side = []
negative_side = []

for i in company:
    
    try:
        name.append(i.find('h2').text.strip())
    except:
        name.append(np.nan)
        
    try:
        rating.append(i.find('span', class_="companyCardWrapper__companyRatingValue").text.strip())
    except:
        rating.append(np.nan)
        
    try:
        reviews.append(i.find_all('span', class_="companyCardWrapper__ActionCount")[0].text.strip())
    except:
        reviews.append(np.nan)
    
    # find_all when two values with same class. Scrap those values with below code
    # first element from same class: for positive
    if len(i.find_all('span', class_="companyCardWrapper__ratingValues"))==2:
        positive_side.append(i.find_all('span', class_="companyCardWrapper__ratingValues")[0].text.strip())
        negative_side.append(i.find_all('span', class_="companyCardWrapper__ratingValues")[1].text.strip())
    else:
        try:
            i.find('span', class_="companyCardWrapper__ratingHeader--high").text.strip()=='Highly Rated For'
            positive_side.append(i.find_all('span', class_="companyCardWrapper__ratingValues")[0].text.strip())
        except:
            positive_side.append(np.nan)

        # second element from same class: for negative
        try:
            i.find('span', class_="companyCardWrapper__ratingHeader--critical").text.strip()=='Critically Rated For'
            negative_side.append(i.find_all('span', class_="companyCardWrapper__ratingValues")[0].text.strip())
        except:
            negative_side.append(np.nan)
        
    if len(i.find('span', class_="companyCardWrapper__interLinking").text.strip().split('|'))==5:
        type.append(i.find('span', class_="companyCardWrapper__interLinking").text.strip().split('|')[0].strip())
        size.append(i.find('span', class_="companyCardWrapper__interLinking").text.strip().split('|')[1].strip())
        public_private.append(i.find('span', class_="companyCardWrapper__interLinking").text.strip().split('|')[2].strip())
        how_old.append(i.find('span', class_="companyCardWrapper__interLinking").text.strip().split('|')[3].strip())
        city.append(i.find('span', class_="companyCardWrapper__interLinking").text.strip().split('|')[4].strip())
    else:
        lst = i.find('span', class_="companyCardWrapper__interLinking").text.strip().split('|')

        try:
            city_ele = [j for j in i.find('span', class_="companyCardWrapper__interLinking").text.strip().split('|') if 'more' in j][0]
            city.append(city_ele.strip())
            lst.remove(city_ele)
        except:
            city.append(np.nan)

        try:
            how_old_ele = [j for j in i.find('span', class_="companyCardWrapper__interLinking").text.strip().split('|') if 'old' in j][0]
            how_old.append(how_old_ele.strip())
            lst.remove(how_old_ele)
        except:
            how_old.append(np.nan)

        try:
            size_ele = [j for j in i.find('span', class_="companyCardWrapper__interLinking").text.strip().split('|') if 'Employees' in j][0]
            size.append(size_ele.strip())
            lst.remove(size_ele)
        except:
            size.append(np.nan)
            
        try:
            type.append(lst[0].strip())
            del lst[0]
        except:
            type.append(np.nan)

        try:
            public_private.append(lst[0].strip())    
        except:
            public_private.append(np.nan)

In [11]:
pd.DataFrame({'Name':name,
              'Ratings':rating,
              'Reviews': reviews,
             'Company Type':type,
             'Company Size':size,
             'Registered':public_private,
             'Old':how_old,
             'City':city,
              'Higly Rated For':positive_side,
              'Critically Rated For':negative_side
             })

,Name,Ratings,Reviews,Company Type,Company Size,Registered,Old,City,Higly Rated For,Critically Rated For
0,TCS,3.8,69.6k,IT Services & Consulting,1 Lakh+ Employees,Public,55 years old,Mumbai +324 more,"Job Security, Work Life Balance","Promotions / Appraisal, Salary & Benefits"
1,Accenture,4.0,44.1k,IT Services & Consulting,1 Lakh+ Employees,Public,34 years old,Dublin +160 more,"Company Culture, Skill Development / Learning,...",NaN
2,Cognizant,3.9,39.7k,IT Services & Consulting,1 Lakh+ Employees,Forbes Global 2000,29 years old,Teaneck. New Jersey. +141 more,Skill Development / Learning,Promotions / Appraisal
3,Wipro,3.8,37.1k,IT Services & Consulting,1 Lakh+ Employees,Public,78 years old,Bangalore/Bengaluru +265 more,Job Security,"Promotions / Appraisal, Salary & Benefits"
4,HDFC Bank,3.9,31.9k,Banking,1 Lakh+ Employees,Public,29 years old,Mumbai +1483 more,"Job Security, Skill Development / Learning",Promotions / Appraisal
5,ICICI Bank,4.0,31.8k,Banking,1 Lakh+ Employees,Public,29 years old,Mumbai +1240 more,"Job Security, Skill Development / Learning, Co...",NaN
6,Infosys,3.9,30.1k,IT Services & Consulting,1 Lakh+ Employees,Public,42 years old,Bengaluru/Bangalore +160 more,"Job Security, Company Culture, Skill Developme...","Salary & Benefits, Promotions / Appraisal"
7,Capgemini,3.8,28k,IT Services & Consulting,1 Lakh+ Employees,Public,56 years old,Paris +112 more,"Job Security, Work Life Balance, Skill Develop...","Promotions / Appraisal, Salary & Benefits"
8,HCLTech,3.7,26.8k,IT Services & Consulting,1 Lakh+ Employees,Public,32 years old,Noida +172 more,Job Security,"Promotions / Appraisal, Salary & Benefits, Wor..."
9,Tech Mahindra,3.7,26.2k,IT Services & Consulting,1 Lakh+ Employees,Public,37 years old,Pune +251 more,NaN,"Promotions / Appraisal, Salary & Benefits"


# Trying to scrap 5 pages: 

In [21]:
headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'}

In [22]:
final = pd.DataFrame()
for j in range(1,6):
    webpage = requests.get(f"https://www.ambitionbox.com/list-of-companies?campaign=desktop_nav&page={j}",headers=headers).text
    soup = BeautifulSoup(webpage,'lxml')
    company = soup.find_all('div', class_="companyCardWrapper")
    
    name = []
    rating = []
    reviews = []
    type = []
    size = []
    public_private = []
    how_old = []
    city = []
    positive_side = []
    negative_side = []
    
    for i in company:

        try:
            name.append(i.find('h2').text.strip())
        except:
            name.append(np.nan)

        try:
            rating.append(i.find('span', class_="companyCardWrapper__companyRatingValue").text.strip())
        except:
            rating.append(np.nan)

        try:
            reviews.append(i.find_all('span', class_="companyCardWrapper__ActionCount")[0].text.strip())
        except:
            reviews.append(np.nan)

        # find_all when two values with same class. Scrap those values with below code
        # first element from same class: for positive
        if len(i.find_all('span', class_="companyCardWrapper__ratingValues"))==2:
            positive_side.append(i.find_all('span', class_="companyCardWrapper__ratingValues")[0].text.strip())
            negative_side.append(i.find_all('span', class_="companyCardWrapper__ratingValues")[1].text.strip())
        else:
            try:
                i.find('span', class_="companyCardWrapper__ratingHeader--high").text.strip()=='Highly Rated For'
                positive_side.append(i.find_all('span', class_="companyCardWrapper__ratingValues")[0].text.strip())
            except:
                positive_side.append(np.nan)

            # second element from same class: for negative
            try:
                i.find('span', class_="companyCardWrapper__ratingHeader--critical").text.strip()=='Critically Rated For'
                negative_side.append(i.find_all('span', class_="companyCardWrapper__ratingValues")[0].text.strip())
            except:
                negative_side.append(np.nan)

        if len(i.find('span', class_="companyCardWrapper__interLinking").text.strip().split('|'))==5:
            type.append(i.find('span', class_="companyCardWrapper__interLinking").text.strip().split('|')[0].strip())
            size.append(i.find('span', class_="companyCardWrapper__interLinking").text.strip().split('|')[1].strip())
            public_private.append(i.find('span', class_="companyCardWrapper__interLinking").text.strip().split('|')[2].strip())
            how_old.append(i.find('span', class_="companyCardWrapper__interLinking").text.strip().split('|')[3].strip())
            city.append(i.find('span', class_="companyCardWrapper__interLinking").text.strip().split('|')[4].strip())
        else:
            lst = i.find('span', class_="companyCardWrapper__interLinking").text.strip().split('|')

            try:
                city_ele = [j for j in i.find('span', class_="companyCardWrapper__interLinking").text.strip().split('|') if 'more' in j][0]
                city.append(city_ele.strip())
                lst.remove(city_ele)
            except:
                city.append(np.nan)

            try:
                how_old_ele = [j for j in i.find('span', class_="companyCardWrapper__interLinking").text.strip().split('|') if 'old' in j][0]
                how_old.append(how_old_ele.strip())
                lst.remove(how_old_ele)
            except:
                how_old.append(np.nan)

            try:
                size_ele = [j for j in i.find('span', class_="companyCardWrapper__interLinking").text.strip().split('|') if 'Employees' in j][0]
                size.append(size_ele.strip())
                lst.remove(size_ele)
            except:
                size.append(np.nan)
            
            try:    
                type.append(lst[0].strip())
                del lst[0]
            except:
                type.append(np.nan)

            try:
                public_private.append(lst[0].strip())    
            except:
                public_private.append(np.nan)
                
    df = pd.DataFrame({'Name':name,
          'Ratings':rating,
          'Reviews': reviews,
         'Company Type':type,
         'Company Size':size,
         'Registered':public_private,
         'Old':how_old,
         'City':city,
          'Higly Rated For':positive_side,
          'Critically Rated For':negative_side
         })
    final = pd.concat([final,df], axis=0, ignore_index=True)
    print(f"page {j} is successfully added to dataframe !!!")

page 1 is successfully added to dataframe !!!
page 2 is successfully added to dataframe !!!
page 3 is successfully added to dataframe !!!
page 4 is successfully added to dataframe !!!
page 5 is successfully added to dataframe !!!


In [23]:
final

,Name,Ratings,Reviews,Company Type,Company Size,Registered,Old,City,Higly Rated For,Critically Rated For
0,TCS,3.8,69.6k,IT Services & Consulting,1 Lakh+ Employees,Public,55 years old,Mumbai +324 more,"Job Security, Work Life Balance","Promotions / Appraisal, Salary & Benefits"
1,Accenture,4.0,44.1k,IT Services & Consulting,1 Lakh+ Employees,Public,34 years old,Dublin +160 more,"Company Culture, Skill Development / Learning,...",NaN
2,Cognizant,3.9,39.7k,IT Services & Consulting,1 Lakh+ Employees,Forbes Global 2000,29 years old,Teaneck. New Jersey. +141 more,Skill Development / Learning,Promotions / Appraisal
3,Wipro,3.8,37.1k,IT Services & Consulting,1 Lakh+ Employees,Public,78 years old,Bangalore/Bengaluru +265 more,Job Security,"Promotions / Appraisal, Salary & Benefits"
4,HDFC Bank,3.9,31.9k,Banking,1 Lakh+ Employees,Public,29 years old,Mumbai +1483 more,"Job Security, Skill Development / Learning",Promotions / Appraisal
...,...,...,...,...,...,...,...,...,...,...
95,Siemens,4.2,4k,Power,5k-10k Employees,Public,176 years old,Munich +121 more,"Job Security, Company Culture, Work Life Balance",NaN
96,Apollo Hospitals,4.1,4k,Healthcare,10k-50k Employees,Public,40 years old,Chennai +176 more,"Job Security, Skill Development / Learning, Co...",NaN
97,JLL,4.2,4k,Real Estate,10k-50k Employees,Forbes Global 2000,26 years old,Mumbai +68 more,"Skill Development / Learning, Job Security, Co...",NaN
98,Vivo,4.0,4k,Consumer Electronics & Appliances,5k-10k Employees,Conglomerate,14 years old,Gurgaon/Gurugram +362 more,"Skill Development / Learning, Company Culture,...",Promotions / Appraisal
